In [56]:
# =========================================================
# 0. Imports and settings
# =========================================================
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
from datetime import datetime


In [ ]:
from google.colab import files
uploaded = files.upload()


In [51]:
import os
print(os.getcwd())


/content


In [50]:
df = pd.read_excel("C:/Users/USER/Documents/Machine_Learning/Deep_Learning/Waqar_hussain/normalized_features.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/USER/Documents/Machine_Learning/Deep_Learning/Waqar_hussain/normalized_features.xlsx'

In [9]:
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [10]:
# Training / validation / test fractions
TEST_FRAC = 0.15          # fraction of full dataset for test
VALID_FRAC = 0.15         # fraction of full dataset for validation


In [11]:
# Use PCA to reduce dimensionality if you have many features and training is slow.
USE_PCA = True
PCA_COMPONENTS = 15      # set to None or <= number of features; tweak as necessary

In [12]:
# Model & training hyperparameters (you can tune these)
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
DROPOUT_RATE = 0.3

In [13]:
# Output artifact paths
MODEL_DIR = "./model_artifacts"
os.makedirs(MODEL_DIR, exist_ok=True)
MODEL_PATH = os.path.join(MODEL_DIR, "dnn_model.h5")
BEST_MODEL_PATH = os.path.join(MODEL_DIR, "dnn_model_best.h5")
ARTIFACTS_PATH = os.path.join(MODEL_DIR, "preprocessing_artifacts.pkl")

In [14]:
# =========================================================
# 1. Environment check: try to import TensorFlow/Keras
# =========================================================
USE_KERAS = False
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, callbacks, models
    print("TensorFlow/Keras detected. Will train using Keras DNN.")
    USE_KERAS = True
except Exception as e:
    print("TensorFlow not available or failed to import. Will fallback to scikit-learn MLP.")
    print("Import error:", e)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

# sklearn fallback
from sklearn.neural_network import MLPClassifier

TensorFlow/Keras detected. Will train using Keras DNN.


In [35]:
# Path to your CSV file (change if needed)
CSV_PATH = 'normalized_features_16.csv'
#r'C:\Users\USER\Documents\Machine Learning\Deep Learning\Waqar_hussain\data\normalized_features_16.csv'

In [36]:
# =========================================================
# 2. Load the dataset
# =========================================================
print(f"\nLoading dataset from: {CSV_PATH}")

# Check that the CSV exists and try to resolve relative paths to give a clear error if not found
if not os.path.exists(CSV_PATH):
	alt_path = os.path.join(os.getcwd(), CSV_PATH)
	if os.path.exists(alt_path):
		CSV_PATH = alt_path
		print(f"Resolved CSV_PATH to absolute path: {CSV_PATH}")
	else:
		raise FileNotFoundError(
			f"CSV file not found at: {CSV_PATH}\n"
			f"Current working directory: {os.getcwd()}\n"
			"Please verify CSV_PATH or provide the correct path to the CSV file."
		)

# Read CSV (explicitly surface errors if they occur)
try:
	df = pd.read_csv(CSV_PATH)
	print("Loaded. Shape:", df.shape)
except Exception as e:
	raise RuntimeError(f"Failed to read CSV at {CSV_PATH}: {e}")


Loading dataset from: normalized_features_16.csv


FileNotFoundError: CSV file not found at: normalized_features_16.csv
Current working directory: /content
Please verify CSV_PATH or provide the correct path to the CSV file.

In [ ]:


print(f"\nLoading dataset from: {CSV_PATH}")
df = pd.read_csv(CSV_PATH)
print("Loaded. Shape:", df.shape)
print("Columns sample (first 30):", df.columns.tolist()[:30])

# =========================================================
# 3. Quick EDA: shape, dtypes, missing values, target guess
# =========================================================
print("\n=== Quick EDA ===")
print("Head (5 rows):")
display_cols = df.columns.tolist()[:40]
print(df.head()[display_cols])

print("\nInfo:")
print(df.info())

print("\nNumeric summary (first 10 rows of .describe()):")
print(df.describe().transpose().head(10))

# Missing values
missing = df.isnull().sum()
missing_nonzero = missing[missing > 0]
if len(missing_nonzero) > 0:
    print("\nColumns with missing values:")
    print(missing_nonzero)
else:
    print("\nNo missing values detected.")

# Identify target column: prefer common names, otherwise last column
candidate_names = ["target", "label", "class", "y", "outcome", "Label"]
target_col = None
for name in candidate_names:
    if name in df.columns:
        target_col = name
        break
if target_col is None:
    target_col = df.columns[-1]
    print(f"No common target names found. Using last column as target: '{target_col}'")
else:
    print(f"Using detected target column: '{target_col}'")

# Show distribution of target
print("\nTarget distribution:")
print(df[target_col].value_counts().sort_index())
print(df[target_col].value_counts(normalize=True).sort_index())

# Visual: target distribution (if not too many classes)
num_classes = df[target_col].nunique()
if num_classes <= 50:
    plt.figure(figsize=(8,4))
    sns.countplot(x=target_col, data=df, order=df[target_col].value_counts().index)
    plt.title("Target class distribution")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()
else:
    print(f"Large number of classes ({num_classes}) — skipping full count plot.")

# =========================================================
# 4. Preprocessing: separate X,y, encode, scale, optional PCA
# =========================================================
print("\n=== Preprocessing ===")
X = df.drop(columns=[target_col]).copy()
y = df[target_col].copy()

print("Feature matrix shape:", X.shape)
print("Target shape:", y.shape)

# Handle non-numeric features in X
non_numeric_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
if non_numeric_cols:
    print("Non-numeric columns detected:", non_numeric_cols)
    # Simple approach: label-encode each column (alternatively: one-hot if few categories)
    for col in non_numeric_cols:
        le_tmp = LabelEncoder()
        X[col] = le_tmp.fit_transform(X[col].astype(str))
    print("Encoded non-numeric feature columns with LabelEncoder.")
else:
    print("All features appear numeric.")

# Scale features (StandardScaler). Even if features were normalized, scaling is safe.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Applied StandardScaler to features.")

# Optional: PCA for dimensionality reduction (speeds training, may lose some info)
use_pca = USE_PCA and (PCA_COMPONENTS is not None)
pca = None
if use_pca:
    from sklearn.decomposition import PCA
    n_features = X_scaled.shape[1]
    pca_n = PCA_COMPONENTS if PCA_COMPONENTS < n_features else min(n_features, PCA_COMPONENTS)
    print(f"Applying PCA: reducing from {n_features} -> {pca_n} components")
    pca = PCA(n_components=pca_n, random_state=RANDOM_STATE)
    X_reduced = pca.fit_transform(X_scaled)
    input_dim = X_reduced.shape[1]
else:
    X_reduced = X_scaled
    input_dim = X_reduced.shape[1]

print("Final input dimension for model:", input_dim)

# Encode the target y
# If target is categorical string or integer categories, LabelEncoder -> [0..C-1]
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.astype(str))
class_names = label_encoder.classes_
n_classes = len(class_names)
print(f"Detected {n_classes} unique classes in target.")

# If user intends binary classification but labels are 2 categories, ensure correct shapes
binary_classification = (n_classes == 2)

# =========================================================
# 5. Train / Validation / Test split (stratified)
# =========================================================
print("\n=== Train / Val / Test split ===")
X_temp, X_test, y_temp, y_test = train_test_split(
    X_reduced, y_encoded, test_size=TEST_FRAC, random_state=RANDOM_STATE,
    stratify=y_encoded if n_classes > 1 else None
)

# compute fraction of temp to allocate to validation so that VALID_FRAC is of original data
val_fraction_of_temp = VALID_FRAC / (1.0 - TEST_FRAC)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=val_fraction_of_temp, random_state=RANDOM_STATE,
    stratify=y_temp if n_classes > 1 else None
)

print("Train shape:", X_train.shape, y_train.shape)
print("Val shape:", X_val.shape, y_val.shape)
print("Test shape:", X_test.shape, y_test.shape)

# =========================================================
# 6. Handle class imbalance (optional)
# =========================================================
print("\n=== Class imbalance handling ===")
class_counts = np.bincount(y_encoded)
print("Class counts:", class_counts)
if class_counts.min() / class_counts.max() < 0.5:
    print("Noticeable class imbalance detected. We'll compute class weights for training.")
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = {i: w for i, w in enumerate(class_weights)}
    print("Computed class_weight dict (example):", dict(list(class_weight_dict.items())[:10]))
else:
    class_weight_dict = None
    print("Class distribution relatively balanced or acceptable.")

# =========================================================
# 7. Build model (Keras DNN if available; else sklearn MLP)
# =========================================================
print("\n=== Model building & training ===")
if USE_KERAS:
    # Build Keras sequential model — configurable architecture
    def build_dnn(input_dim, n_classes, dropout_rate=DROPOUT_RATE, lr=LEARNING_RATE):
        model = models.Sequential()
        model.add(layers.Input(shape=(input_dim,)))
        # Hidden layers — adjust sizes as needed
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dropout(dropout_rate))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dropout(dropout_rate))
        model.add(layers.Dense(64, activation='relu'))
        # Output
        if n_classes == 2:
            model.add(layers.Dense(1, activation='sigmoid'))
            loss = 'binary_crossentropy'
            metrics = ['accuracy']
        else:
            model.add(layers.Dense(n_classes, activation='softmax'))
            loss = 'categorical_crossentropy'
            metrics = ['accuracy']
        opt = keras.optimizers.Adam(learning_rate=lr)
        model.compile(optimizer=opt, loss=loss, metrics=metrics)
        return model

    model = build_dnn(input_dim=input_dim, n_classes=n_classes)
    model.summary()

    # Prepare y for Keras
    if n_classes == 2:
        y_train_keras = np.array(y_train)
        y_val_keras = np.array(y_val)
    else:
        from tensorflow.keras.utils import to_categorical
        y_train_keras = to_categorical(y_train, num_classes=n_classes)
        y_val_keras = to_categorical(y_val, num_classes=n_classes)

    # Callbacks
    es = callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
    mc = callbacks.ModelCheckpoint(BEST_MODEL_PATH, monitor='val_loss', save_best_only=True, verbose=1)

    # Train with or without class weights
    fit_kwargs = dict(
        x=X_train, y=y_train_keras,
        validation_data=(X_val, y_val_keras),
        epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[es, mc], verbose=2
    )
    if class_weight_dict is not None:
        fit_kwargs["class_weight"] = class_weight_dict

    history = model.fit(**fit_kwargs)

    # Save final model
    model.save(MODEL_PATH)
    print(f"Saved final model to {MODEL_PATH}")
    print(f"Best model (checkpoint) saved to {BEST_MODEL_PATH}")

    # Evaluate on test set
    if n_classes == 2:
        y_test_prob = model.predict(X_test).ravel()
        y_test_pred = (y_test_prob >= 0.5).astype(int)
    else:
        y_test_prob = model.predict(X_test)
        y_test_pred = np.argmax(y_test_prob, axis=1)

else:
    # sklearn MLP fallback
    print("Keras not available — using sklearn MLPClassifier (may be slower for many classes/features).")
    mlp = MLPClassifier(hidden_layer_sizes=(256,128,64), activation='relu', solver='adam',
                        batch_size=min(128, X_train.shape[0]), learning_rate_init=LEARNING_RATE,
                        max_iter=200, early_stopping=True, n_iter_no_change=10, random_state=RANDOM_STATE, verbose=True)

    if class_weight_dict is not None:
        # sklearn MLP doesn't accept class_weight directly for MLPClassifier — you'd need other models (e.g., balanced subsampling)
        print("Note: sklearn MLPClassifier does not accept class_weight natively. Consider resampling or using a different classifier.")
    mlp.fit(X_train, y_train)
    # Save model with joblib
    try:
        import joblib
        joblib.dump(mlp, os.path.join(MODEL_DIR, "mlp_sklearn.joblib"))
    except Exception as e:
        print("Could not save sklearn model via joblib:", e)
    y_test_pred = mlp.predict(X_test)

# =========================================================
# 8. Evaluation: accuracy, classification report, confusion matrix, macro F1
# =========================================================
print("\n=== Evaluation on test set ===")
accuracy = accuracy_score(y_test, y_test_pred)
macro_f1 = f1_score(y_test, y_test_pred, average='macro')
print(f"Test accuracy: {accuracy:.4f}")
print(f"Test macro F1: {macro_f1:.4f}")

print("\nClassification report (per-class precision/recall/f1):")
print(classification_report(y_test, y_test_pred, target_names=[str(c) for c in class_names], zero_division=0))

# Confusion matrix (may be large)
cm = confusion_matrix(y_test, y_test_pred)
print("Confusion matrix shape:", cm.shape)

# Visualize confusion matrix for small number of classes
if n_classes <= 30:
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.ylabel('True')
    plt.xlabel('Predicted')
    plt.title('Confusion matrix')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
else:
    print("Skipping confusion matrix heatmap (too many classes).")

# =========================================================
# 9. Plot training curves (if Keras used)
# =========================================================
if USE_KERAS and 'history' in globals():
    h = history.history
    # Accuracy curve
    plt.figure()
    if 'accuracy' in h:
        plt.plot(h['accuracy'], label='train_accuracy')
        plt.plot(h.get('val_accuracy', []), label='val_accuracy')
        plt.title("Accuracy")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.grid(True)
        plt.show()
    # Loss curve
    plt.figure()
    plt.plot(h['loss'], label='train_loss')
    plt.plot(h.get('val_loss', []), label='val_loss')
    plt.title("Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

# =========================================================
# 10. Save preprocessing artifacts for later inference
# =========================================================
artifacts = {
    "scaler": scaler,
    "label_encoder": label_encoder,
    "pca": pca,
    "input_dim": input_dim,
    "class_names": list(class_names),
    "trained_on": str(datetime.utcnow())
}
with open(ARTIFACTS_PATH, "wb") as f:
    pickle.dump(artifacts, f)
print(f"Saved preprocessing artifacts to: {ARTIFACTS_PATH}")

# For convenience, also dump a small JSON summary
summary = {
    "n_samples": int(df.shape[0]),
    "n_features_original": int(X.shape[1]),
    "n_features_final": int(input_dim),
    "n_classes": int(n_classes),
    "model_path": MODEL_PATH if USE_KERAS else os.path.join(MODEL_DIR, "mlp_sklearn.joblib"),
    "artifacts_path": ARTIFACTS_PATH
}
with open(os.path.join(MODEL_DIR, "run_summary.json"), "w") as f:
    json.dump(summary, f, indent=2)
print("Wrote run summary to model_artifacts/run_summary.json")

# =========================================================
# 11. Quick inference example (how to use saved artifacts & model)
# =========================================================
print("\n=== Quick inference example (code snippet) ===")
print("""
# Example: Load artifacts and make predictions on new_data (pandas DataFrame with same columns as original X)
import pickle
import numpy as np
from tensorflow import keras  # if using Keras

artifacts = pickle.load(open("model_artifacts/preprocessing_artifacts.pkl", "rb"))
scaler = artifacts['scaler']
label_encoder = artifacts['label_encoder']
pca = artifacts.get('pca', None)

# new_df: a pandas DataFrame row(s)
X_new = new_df.copy()
# apply same preprocessing: label encoding for categorical columns (you must reproduce same mapping), scaling, PCA (if used)
X_new_scaled = scaler.transform(X_new.values)
if pca is not None:
    X_new_reduced = pca.transform(X_new_scaled)
else:
    X_new_reduced = X_new_scaled

# load model and predict
model = keras.models.load_model("model_artifacts/dnn_model_best.h5")
y_prob = model.predict(X_new_reduced)
if y_prob.shape[1] == 1:  # binary
    preds = (y_prob.ravel() >= 0.5).astype(int)
else:
    preds = y_prob.argmax(axis=1)
pred_labels = label_encoder.inverse_transform(preds)
print(pred_labels)
""")

# =========================================================
# 12. Final notes and suggestions
# =========================================================
print("\n=== Final notes & suggestions ===")
print("""
- If you have many features (thousands), PCA or feature selection can speed training and reduce overfitting.
- For heavy tabular tasks, tree-based models (RandomForest, XGBoost, LightGBM, CatBoost) often outperform simple DNNs.
- If classes are imbalanced, options are:
   * Use class weights (we computed them above for Keras),
   * Resample: oversample minority (SMOTE) or undersample majority,
   * Use focal loss (for keras) for extreme imbalance.
- For hyperparameter tuning: try RandomizedSearchCV / KerasTuner to tune number of layers, units, dropout, lr, batch_size.
- If GPU is available, Keras will use it (ensure proper TF / CUDA installation).
- If training is slow / OOM: reduce batch size, reduce model size, or reduce PCA components.
""")

print("\nPipeline complete. Check the 'model_artifacts' folder for saved model and preprocessing files.")
